<a href="https://colab.research.google.com/github/Isaac-Gregory/House-Pricing-SWD/blob/main/Copy_of_housepricing_noiseanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Isaac Gregory and Bibek Lamsal

**House Pricing** *for SWD Course*

Noisy Feature Analysis

##Imports and Dataset

In [5]:
from google.colab import files
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import copy as cp
from scipy.stats import chi2_contingency

In [4]:
# # Import kaggle.json file
# files.upload()
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [3]:
# Load dataset
# !kaggle competitions download -c house-prices-advanced-regression-techniques

In [2]:
# Unzipping data from file
# !unzip house-prices-advanced-regression-techniques.zip

In [6]:
# Reading in the CSV
data = pd.read_csv('train.csv')
data.drop(['Id'], axis=1, inplace=True)
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


##Preprocessing

##Noise Analysis

###Additional Noise Analysis

In [15]:
# prompt: remove SalesPrice from X_train adn save it in y_train,split categorical and numerical feature from X_train data. now find p value of all features in cat_cols as compared to y_train, from x_train remove all features with higher p value than 0.05

import pandas as pd
# Separate labels from samples
y_train = data['SalePrice']
X_train = data.drop(['SalePrice'], axis=1)

# Split numerical from categorical features
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()
num_cols = X_train.select_dtypes(include=['number']).columns.tolist()

# Create copies of lists for tracking
cat_cols_copy = cp.deepcopy(cat_cols)

# Dictionaries for tracking results
cat_corr = {}

# Use chi-squared analysis on categorical features
for col1 in cat_cols:
  # Contingency table
  contingency_tbl = pd.crosstab(X_train[col1], y_train)

  # Ensure table has values (i.e. is not empty)
  if contingency_tbl.size == 0:
    # Assigning default values
    cat_corr[col1] = (0, 1)
    continue

  # Perform chi-squared analysis
  chi, p, dof, expected = chi2_contingency(contingency_tbl)

  # Save results
  cat_corr[col1] = (chi, p)

# Finding low p-values
to_remove_cat = []
for key in cat_corr:
  if cat_corr[key][1] > 0.05:
    print(key, cat_corr[key])
    to_remove_cat.append(key)

# # Remove features with p-value greater than 0.05
# X_train = X_train.drop(to_remove, axis=1)


Alley (91.00000000000001, 0.20972415885759163)
LandContour (2072.335508433759, 0.08674645041917711)
Utilities (242.49942883253368, 1.0)
LandSlope (1388.847891459522, 0.10508638737793884)
Condition1 (4147.618513410399, 1.0)
Condition2 (4772.610202894631, 0.07598640644469955)
BldgType (2354.0254509759116, 0.9999860714473023)
HouseStyle (4596.789414569782, 0.6482615179447816)
RoofStyle (2665.6801874818393, 1.0)
RoofMatl (3146.4668880631225, 1.0)
Exterior1st (8712.732778251357, 0.9999839433628513)
Exterior2nd (9785.82598370394, 0.8469189757654344)
MasVnrType (781.6277258895132, 0.1519389355542077)
BsmtFinType1 (3077.080134851721, 0.9728950558884571)
BsmtFinType2 (2851.850846021678, 0.9999995053342521)
HeatingQC (2411.0596566165887, 0.9995946871632676)
Electrical (2764.8705958910587, 0.055687991437252254)
Functional (3073.9216679397996, 1.0)
GarageType (3284.2615366284517, 0.1324830638502916)
GarageCond (2267.78725969041, 0.999985676535125)
PavedDrive (1204.774866369986, 0.9912799626956017)

In [16]:
# prompt: remove SalesPrice from X_train adn save it in y_train,split categorical and numerical feature from X_train data. now find correlation of all features in numerical data as compared to y_train, from x_train remove all features with low correlation from X_train

# Separate labels from samples
y_train = data['SalePrice']
X_train = data.drop(['SalePrice'], axis=1)

# Split numerical from categorical features
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()
num_cols = X_train.select_dtypes(include=['number']).columns.tolist()

# Find and graph correlation between all features and the labels
targ_corr = {}
for feature in num_cols:
  # Calculate correlation
  pearson_temp = X_train[feature].corr(y_train)
  spearman_temp = X_train[feature].corr(y_train, method='spearman')
  # Save correlation into array
  targ_corr[feature] = (pearson_temp,spearman_temp)

# Observe features and remove using threshold on array
noisy_features_num = []
for feature in targ_corr:
  if(abs(targ_corr[feature][0]) < 0.3 and abs(targ_corr[feature][1]) < 0.3):
    print(feature, targ_corr[feature])
    noisy_features_num.append(feature)

# Remove features with low correlation
X_train = X_train.drop(noisy_features_num, axis=1)
X_train = X_train.drop(to_remove_cat, axis=1)

X_train.to_excel('X_train.xlsx', index=False)




MSSubClass (-0.08428413512659531, 0.007192252911733476)
OverallCond (-0.07785589404867803, -0.12932494660061317)
BsmtFinSF2 (-0.011378121450215125, -0.03880613204589418)
BsmtUnfSF (0.21447910554696892, 0.185196629420762)
LowQualFinSF (-0.02560613000067959, -0.06771915407896568)
BsmtFullBath (0.22712223313149382, 0.22512486719612368)
BsmtHalfBath (-0.016844154297359016, -0.012188876310787316)
BedroomAbvGr (0.16821315430073988, 0.23490671789027862)
KitchenAbvGr (-0.13590737084214122, -0.1648257549850205)
EnclosedPorch (-0.12857795792595653, -0.2183936205521982)
3SsnPorch (0.04458366533574846, 0.06544021620062833)
ScreenPorch (0.11144657114291105, 0.1000697202012266)
PoolArea (0.09240354949187321, 0.058452996689891755)
MiscVal (-0.02118957964030325, -0.0627270024962966)
MoSold (0.046432245223819384, 0.06943224370457042)
YrSold (-0.028922585168730378, -0.029899134912615286)
